In [ ]:
# -*- coding: utf-8 -*-
from keras.preprocessing import image              #加载包
import cv2
import keras
from keras import backend as K
from keras.models import load_model
#from PIL import Image
import numpy as np

from keras.models import Model

In [50]:
#第一步：加载测试图片
img_path = "./4-1.png"                                  #测试图片路径
img = image.load_img(img_path,target_size=(28,28))      #cv2加载测试图片reshape
img = img.convert('L')                                  #测试图片转为灰度值

x = image.img_to_array(img)             #图像数据转为数组
print(x.shape)                          #打印图像的维度尺寸
x = np.expand_dims(x, axis=0)           #图像周围补0
x=x.astype('float32')/255.              #图像像素归一化


(28, 28, 1)


In [51]:
#第二步：导入已经训练的模型
model=keras.models.load_model('./model1.h5')
#模型概括打印
print("模型概括：")
model.summary()
#模型的层数
print("网络层数：",len(model.layers))
#模型包括的层名词
print("每一层的名称：",model.layers)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1568)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1568)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [52]:
#第三步，将模型作为一个层，输出第0层的输出
layer_model = Model(inputs=model.input, outputs=model.layers[0].output)

In [5]:
#第四步，调用新建的层模型的predict方法，得到模型的输出
feature=layer_model.predict(x)
print(feature.dtype)                 #打印输出特征图
print(feature.shape)           #打印输出特征图的形状
print(type(feature))           #打印输出特征图的数据类型

feature=feature.transpose(3,0,1,2)          #转置
print(feature.shape)
print("索引值：{0:.7f}".format(feature[0,0,0,4]))
print(feature.astype("float32"))                 #打印转置后的输出特征图

float32
(1, 28, 28, 16)
<class 'numpy.ndarray'>
(16, 1, 28, 28)
索引值：0.0000000
[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]]


 [[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]]


 [[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]]


 ...


 [[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]]


 [[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]]


 [[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 

In [1]:
#方法二===============================================================================================================================
# -*- coding: utf-8 -*-
from keras.preprocessing import image
import cv2
from keras import backend as K
from keras.models import load_model
#from PIL import Image
import numpy as np

def get_layer_output(model, x, index=-1):
    """
    get the computing result output of any layer you want, default the last layer.
    :param model: primary model
    :param x: input of primary model( x of model.predict([x])[0])
    :param index: index of target layer, i.e., layer[23]
    :return: result
    """
    layer = K.function([model.input], [model.layers[index].output])
    return layer([x])[0]

model = load_model('./model1.h5')              #导入训练好的模型
img_path = "./4-1.png"                         #导入测试图片
img = image.load_img(img_path,target_size=(28,28))      
img = img.convert('L')                         #测试图片转为灰度值

x = image.img_to_array(img)                    #
x = np.expand_dims(x, axis=0)
input_x=x.astype('float32')/255.
conv1 = get_layer_output(model, input_x, index=0)         #提取conv1层的输出
print("conv1层输出原本的形状：",conv1.shape)
#print(conv1)
#conv_1=conv1.reshape(16,1,28,28)
conv_1=conv1.transpose(3,0,1,2)
print("conv1层输出转置后的形状：",conv_1.shape)
print(conv_1.size)
#print(conv_1)
conv_1.tofile('./layer_out/conv1.txt', sep=' ',format='%f')
print("提取conv1输出")

Using TensorFlow backend.
/usr/local/python3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/python3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/python3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/python3/lib/python3.7/site-packages/tensorflow/python/fra

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
conv1层输出原本的形状： (1, 28, 28, 16)
conv1层输出转置后的形状： (16, 1, 28, 28)
12544
提取conv1输出


In [2]:
pool1 = get_layer_output(model, input_x, index=1)
print(pool1.shape)
#print(pool1)

pool_1=pool1.transpose(3,0,1,2)
print(pool_1.shape)
print(pool_1.size)
pool_1.tofile('./layer_out//pool1.txt', sep=' ',format='%f')
#print(pool_1)
print("提取pool1输出")

(1, 14, 14, 16)
(16, 1, 14, 14)
3136
提取pool1输出


In [3]:
conv2 = get_layer_output(model, input_x, index=2)
print(conv2.shape)
conv_2=conv2.transpose(3,0,1,2)
print(conv_2.shape)
print(conv_2.size)
conv_2.tofile('./layer_out//conv2.txt', sep=' ',format='%f')
print("提取conv2输出")

(1, 14, 14, 32)
(32, 1, 14, 14)
6272
提取conv2输出


In [4]:
pool2 = get_layer_output(model, input_x, index=3)
print(pool2.shape)
pool_2=pool2.transpose(3,0,1,2)
print(pool_2.shape)
print(pool_2.size)
pool_2.tofile('./layer_out//pool2.txt', sep=' ',format='%f')
print("提取pool2输出")

(1, 7, 7, 32)
(32, 1, 7, 7)
1568
提取pool2输出


In [5]:
flatten1 = get_layer_output(model, input_x, index=4)
print(flatten1.shape)
print(flatten1.size)
flatten1.tofile('./layer_out//flatten.txt', sep=' ',format='%f')
print("提取flatten1输出")

(1, 1568)
1568
提取flatten1输出


In [6]:
fc1 = get_layer_output(model, input_x, index=6)
print(fc1.shape)
print(type(fc1))
print(fc1.size)
fc1.tofile('./layer_out//fc1.txt', sep=' ',format='%f')
print("提取fc1输出")

(1, 128)
<class 'numpy.ndarray'>
128
提取fc1输出


In [7]:
fc2 = get_layer_output(model, input_x, index=8)
print(fc2.shape)
print(fc2.size)
fc2.tofile('./layer_out//fc2.txt', sep=' ',format='%f')
print("提取fc2输出")
print(fc2)     #输出最后的结果
print(fc2[0][0])     #输出最后的结果
print(type(fc2[0][0]))    #输出最后的结果

(1, 10)
10
提取fc2输出
[[5.2318823e-14 1.3150577e-04 6.0465538e-08 2.3893253e-14 9.9985194e-01
  1.5161897e-13 9.1620738e-09 1.6353997e-05 4.3253381e-10 1.5114437e-12]]
5.2318823e-14
<class 'numpy.float32'>
